<a href="https://colab.research.google.com/github/mayait/ClaseAnalisisDatos/blob/main/nlp/nlp_2023_Twitter_Data_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Nombre del estudiante
Estudiante = "" #@param {type:"string"}
Código = "" #@param {type:"string"}



# "Datascraping con Twitter"


Uno de los superpoderes de la AI/ML es que nos permite entender el mundo desde la perspectiva de los datos.

Twitter es una plataforma que proporciona datos extensos en lenguaje natural, en base a los cuales se puede explorar mucho.

In [1]:
!pip install tweepy textblob wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# importing libraries
import pandas as pd
import tweepy
import time
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

Para acceder a la API de Twitter, necesitaremos generar las claves. 
Estas claves se encuentran en la configuración de la aplicación de Twitter en la pestaña Claves y tokens de acceso:

[ https://apps.twitter.com/app/new]( https://apps.twitter.com/app/new)

In [3]:
#@title Credenciales de Twitter
TWITTER_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPY7WQEAAAAAWoi%2Bliw1TnD6deNUKnCA0KJSU8Q%3DQY62dMjmxlE7mnu2WED6PnQgoax56QV40ZJeFhb1yHvQ7yrR36"
client = tweepy.Client(TWITTER_BEARER_TOKEN)

### Tweepy nos permite conectarnos directamente al API de Twitter y obtener tweets
En este ejemplo nos decargamos 10 Tweets de Ecuador

In [7]:

# La función search_recent_tweets() devuelve un objeto de tipo Response, que contiene los tweets en su atributo data.
# El parámetro max_results indica el número máximo de tweets que se devolverán en la respuesta.

response = client.search_recent_tweets("Young Sheldon -is:retweet lang:en", max_results=100)
tweets = response.data

# Iterar a través de los tweets e imprimir el texto
for tweet in response.data:
    print(tweet.text)



someone needs smack young sheldon once in a while
O tiktok me convenceu a ver young sheldon
feeling like young Sheldon
Please where can I watch Young Sheldon? It's my new feel good movie
@Lashaiwashere Honestly!!! But all I keep remembering is that he dies when Sheldon is young. So I'm waiting
he's either gotta be real young like young sheldon young or old as fuck on his deathbed. anything in between is cowardice https://t.co/0r8YSwuhlb
My fave character rn is Young Sheldon😭😭
Install this app, Watch and download "Young Sheldon" at no cost. https://t.co/RPGhqkTd9T https://t.co/PBMwctnihG
Young Sheldon : Season 3, Meemaw is having a date with Missy's coach https://t.co/cGHeb7wNQu via @YouTube I love this episode that Annie pots played in
Just finished (for now, it's still running) watching Young Sheldon and tried out The Big Bang Theory.. Damn, that shit is uncomfortable, how was that a hit show??
if i was american i believe i would talk like that do in young sheldon
Today I sobbed viole

### Tambien podemos buscar datos de un usuario especifico
Necesitamos el ID del usuario, puedes buscarlo aquí

https://tweeterid.com/

@usfq_ecuador => 66436490

In [8]:
id = '479831176'
tweets = client.get_users_tweets(id=id, tweet_fields=['context_annotations','created_at','geo'], max_results=10)
for tweet in tweets.data:
    print(tweet.created_at, tweet.text)

2022-04-07 22:43:37+00:00 RT @ShaparrithaCor1: @nayibbukele Señor presidente.muy buen trabjo lo q esta asiendo le.hago un llamada para q vallan a cuscatlan en el can…
2022-03-04 15:00:42+00:00 @fco1993 @clubdeviernes No sería más apropiado raterismo
2022-01-15 13:32:28+00:00 @Meridian_Ec @Viteri_BelenEc No odies, eso te enferma y enferma a tus personas cercanas. El odio nace de la envidia ¿o es al revés?
2021-12-31 22:15:46+00:00 De “derecha a izquierda, pasando por el llamado centro”: ladrones, gastando la plata que no es suya y empobreciendo a los que todavía no nacieron(!). https://t.co/ZuzBRp15RG
2021-12-25 22:17:56+00:00 @FreddyP58193593 El socialista basa su existencia en la pobreza y la desigualdad, creando resentimiento, fomentando el odio y alimentando la envidia.
2021-12-25 22:13:44+00:00 @GinaLoor1 Difícil o imposible en un grupo social lleno de complejos, resentimientos y envidia que se dice cristiano y, peor, luego de 10 años de fomentar el odio como política nacional.
202

## Una función que extrae Tuits de un tema y los pasa a un DataFrame

In [19]:
# Función para obtener tweets recientes

def getTweets(keyword, num_tweets):
    # Consulta de búsqueda que incluye el término de búsqueda, excluye retweets y selecciona tweets en español
    query = f'{keyword} -is:retweet lang:en'

    # Uso del paginador de Tweepy para obtener más de 100 tweets recientes a través de la API de Twitter
    tweets = []
    for tweet in tweepy.Paginator(client.search_recent_tweets,
                                  query=query,
                                  tweet_fields=['id', 'created_at', 'public_metrics', 'text', 'source'],
                                  max_results=100).flatten(limit=num_tweets):

        tweets.append(tweet)

    result = []

    # Función para desempaquetar la lista de tweets en un DataFrame
    for tweet in tweets:
        result.append({'id': tweet.id,
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'source': tweet.source,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                       })

    # Crear un DataFrame y convertir la columna 'created_at' a objetos datetime
    df = pd.DataFrame(result)
    df['created_at'] = pd.to_datetime(df['created_at'])
    # Convert timezone-aware datetimes to timezone-naive datetimes
    df['created_at'] = df['created_at'].dt.tz_convert(None)     
    
    # Devolver la lista de tweets
    return df


In [23]:
tweets_claro = getTweets("usfq", 1000)

In [24]:
tweets_claro

,id,text,created_at,source,retweets,replies,likes,quote_count
0,1646560771420061704,#GoldenNeocom2023#NeocomGolden2023 #NeocomGold...,2023-04-13 17:07:34,None,0,0,0,0
1,1646545812342685697,@USFQ_Ecuador @SebasPalaciosSP @dajomesneisi C...,2023-04-13 16:08:07,None,0,0,0,0
2,1646539753347588096,"neocomgolden2023, #usfq, @COCOAUSFQ",2023-04-13 15:44:03,None,0,0,0,0
3,1646193465976406030,Apply and come spend your @MSCActions Postdoct...,2023-04-12 16:48:01,None,1,0,3,0
4,1646139750833684480,"If interested, #Tiputini and @USFQ_Ecuador can...",2023-04-12 13:14:35,None,0,0,0,0
5,1646133659076837378,🚀 Wishing good luck to the HOUN - @CayetanoHer...,2023-04-12 12:50:22,None,3,0,4,0
6,1645892298775973890,@CaicedoLab @ISCTglobal @ISCT_ESP @MaroonKhour...,2023-04-11 20:51:17,None,0,0,3,0
7,1645887041756884998,Joining a scientific community is crucial for ...,2023-04-11 20:30:24,None,5,1,14,0
8,1645405150213730306,🚀 Wishing good luck to the OstheoGlove - @ucat...,2023-04-10 12:35:32,None,3,0,4,0
9,1645203980434436096,@RichardAlbert @UTexasLaw @ccpconstitute @cons...,2023-04-09 23:16:10,None,0,0,1,0


# Ejercicio
Realiza una busqueda en twitter de un termino en español y otro en inglés, exportalos a Excel en archivos separados.

nlp/2022_Twitter_Data_Scrapper.ipynb

> Bloque con sangría

